# GAN Auction Model Explore
#### Date: May-2018
author: O. Osoba  
Exploring GANs with Expert Panel of Discriminators

> tensorboard --logdir=gAuction_1:'./log/train_gan_auction_worker_1'

In [ ]:
#from __future__ import division, print_function #, absolute_import
import argparse, sys, importlib

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint as pp

In [ ]:
# ML libs
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import gan_auction as GA

%pylab inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7,7)

print("Tensorflow version:", tf.__version__)
print("GAN Auctions version:", GA.__version__)

In [ ]:
# Import MNIST data: Keras load if base-tf deprecated
# mnist = tf.keras.datasets.mnist.load_data()
# train = mnist[0][0] #
# train.shape #train[0]

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../ML-Examples/MNIST/", one_hot=True)
(mnist.train.next_batch(1))[0].shape

In [ ]:
# Training Params
num_steps = 3*int(10**4)
lr = (0.0002, 0.0002)

In [ ]:
importlib.reload(GA)
tf.reset_default_graph()

In [ ]:
mgan = GA.GANAuction(data = mnist.train.images,
                     discSpec = (256,),
                     genSpec = (256,),
                     learning_rates = lr,
                     aggregator = "min"
                    )

all_vars = (tf.trainable_variables())
pp.pprint(all_vars)
#mgan.gen_vars
#mgan.disc_vars

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # Initialize the variables (i.e. assign their default value)
sv = tf.train.Saver(max_to_keep=1)

# mgan.auction.shape

In [ ]:
mgan.work(sess, saver = sv, num_epochs = num_steps, batch_sz = 512)

In [ ]:
pp.pprint(mgan.disc_vars)

In [ ]:
# Testing
# Generate images from noise, using the generator network.
n = 6
canvas = np.empty((28 * n, 28 * n))
for i in range(n):
    # Noise input.
    z = np.random.normal(size=[n, mgan.n_size])
    # Generate image from noise.
    g = sess.run(mgan.gen_sample, feed_dict={mgan.n_input: z})
    # Reverse colours for better display
    g = -1 * (g - 1)
    for j in range(n):
        # Draw the generated digits
        canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()

In [ ]:
plt.imshow(g[0].reshape([28, 28]), origin="upper", cmap="gray")

In [ ]:
plt.plot(mgan.g_losses) 
plt.plot(mgan.d_losses)